# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)

#### Grupo:
  - Ranulfo Mascari Neto
  - Heitor Rodrigues Sabino

## **Implementação 1: Maximizar a Recompensa Esperada**
Nesta abordagem, o objetivo é calcular a **melhor estratégia para maximizar a recompensa esperada** ao longo do jogo. Cada estado do jogo é avaliado considerando:
- **Parar o jogo** e receber a recompensa acumulada.
- **Responder à questão atual**, com ou sem o uso de ajudas, avaliando as recompensas esperadas nos estados seguintes.

### **Inputs**
1. **$N$**: Número total de estágios no jogo (16, considerando 15 questões + estágio final).
2. **$r_k$**: Lista das recompensas acumuladas ao parar após cada questão.
3. **$r_k^*$**: Lista das recompensas garantidas ao errar após alcançar os marcos de segurança.
4. **$p_k^*$**: Função que calcula a probabilidade de acertar a questão $k$ sem usar ajudas.
5. **Lifelines**: Lista das ajudas disponíveis, como `["50:50", "phone", "audience"]`.
6. **$c_k^i$**: Fatores de melhoria para cada ajuda na questão $k$.

### **Recorrência**
Para cada questão $k$, as possíveis ações são avaliadas:

1. **Parar o jogo**:
   $$
   \text{reward_stop} = r_k
   $$

2. **Responder à questão**:
   $$
   \text{reward_answer} = p_k \cdot f(k+1, \text{new_state}) + (1 - p_k) \cdot r_k^*
   $$

A recompensa esperada para cada estado $s$ é:
$$
f(k, s) = \max(\text{reward_stop}, \text{reward_answer})
$$

In [1]:
import numpy as np

# r_k represents the rewards for answering each question correctly and choosing to stop at that question
r_k = [0, 150, 300, 450, 900, 1800, 2100, 2700, 3600, 4500, 9000, 18000, 36000, 72000, 144000, 300000]

# r_k_star represents the guaranteed rewards if the contestant fails on a question, returning to the last milestone
r_k_star = [0, 0, 0, 0, 0, 1800, 1800, 1800, 1800, 1800, 9000, 9000, 9000, 9000, 9000, 300000]

# Lifelines available to the contestant: 50:50, phone, and audience
lifelines = ["50:50", "phone", "audience"]

# c_k^i represents the improvement factor for each lifeline at each question
c_k = {
    "50:50": [0, 0.672, 0.698, 0.707, 0.711, 0.714, 0.716, 0.717, 0.718, 0.719, 0.719, 0.720, 0.720, 0.721, 0.721],
    "phone": [0, 0.527, 0.547, 0.554, 0.557, 0.559, 0.561, 0.562,  0.563, 0.563, 0.564, 0.564, 0.564, 0.565, 0.565],
    "audience": [0, 0.745, 0.773, 0.783, 0.788, 0.791, 0.793, 0.795,  0.796, 0.796, 0.797, 0.798, 0.798, 0.799, 0.799]
}

# Total number of stages, including the victory stage
N = 16
# Number of lifelines
m = len(lifelines)

# p_k_star represents the probability of answering a question correctly without using any lifelines
def p_k_star(k):
    return max(0, 0.996 - 0.051 * (k - 1))

# adjust_probability adjusts the probability of answering correctly when using specific lifelines
def adjust_probability(p, used_lifelines, k):
    if not used_lifelines:
        return p
    improvement_factor = np.prod([c_k[lifeline][k] for lifeline in used_lifelines])
    return 1 - (1 - p) * improvement_factor

# expected_rewards is a dynamic programming table to store the maximum expected rewards at each state
expected_rewards = np.zeros((N, 2**m))

# Initialize the victory stage. The maximum reward at stage N-1 is the guaranteed reward for correctly answering all questions
for state in range(2**m):
    expected_rewards[N - 1, state] = r_k_star[N - 1]

# Backward induction: calculate expected rewards for each stage and state
for stage in range(N - 1, 0, -1):
    k = stage - 1
    for state in range(2**m):
        # Calculate the reward if the contestant chooses to stop
        reward_stop = r_k[k]

        # Calculate the reward if the contestant answers without using any lifelines
        p_no_lifeline = p_k_star(stage)
        reward_answer = (
            p_no_lifeline * expected_rewards[stage, state] +
            (1 - p_no_lifeline) * r_k_star[k]
        )

        # Determine which lifelines are available in the current state
        available_lifelines = [lifelines[i] for i in range(m) if (state >> i) & 1]

        # If lifelines are available, calculate the reward using each lifeline
        if available_lifelines:
            p_with_lifeline = adjust_probability(p_no_lifeline, available_lifelines, k)
            new_state = state & ~(sum(1 << i for i, lifeline in enumerate(lifelines) if lifeline in available_lifelines))
            reward_answer = max(
                reward_answer,
                p_with_lifeline * expected_rewards[stage, new_state] +
                (1 - p_with_lifeline) * r_k_star[k]
            )

        # Update the expected reward for the current state
        expected_rewards[k, state] = max(reward_stop, reward_answer)


In [2]:
# @title
# Comparing results with Table 3 from the article
print("\nComparison with Table 3 (Expected Rewards for Question 15)\n")
print("State                | f(State)")
print("---------------------|-----------------")

# Iterate over all possible terminal states
for l1 in [1, 0]:
    for l2 in [1, 0]:
        for l3 in [1, 0]:
            state = (l3 << 2) | (l2 << 1) | l1
            expected_reward = expected_rewards[14, state]
            print(f"15,{l1},{l2},{l3:<1}             | {expected_reward:,.1f}")



Comparison with Table 3 (Expected Rewards for Question 15)

State                | f(State)
---------------------|-----------------
15,1,1,1             | 231,993.9
15,1,1,0             | 214,886.0
15,1,0,1             | 179,635.2
15,1,0,0             | 149,355.7
15,0,1,1             | 205,678.1
15,0,1,0             | 181,950.0
15,0,0,1             | 144,000.0
15,0,0,0             | 144,000.0


## **Implementação 2: Maximizar a Probabilidade de Alcançar uma Questão**
Nesta abordagem, o objetivo é calcular a **estratégia ótima para maximizar a probabilidade de alcançar e responder corretamente uma questão alvo** $w$. Cada estado do jogo é avaliado para determinar a melhor sequência de ações que maximize as chances de sucesso.

### **Inputs**
1. **$w$**: Número da questão alvo.
2. **$N$**: Número total de estágios no jogo (16).
3. **$p_k^*$**: Função que calcula a probabilidade de acertar a questão $k$ sem ajudas.
4. **Lifelines**: Lista das ajudas disponíveis, como `["50:50", "phone", "audience"]`.
5. **$c_k^i$**: Fatores de melhoria para cada ajuda na questão $k$.

### **Recorrência**
1. **Condição Base**:
   Se $k = w$, a probabilidade de sucesso é $1$:
   $$
   f(w, \text{state}) = 1
   $$

2. **Estados Anteriores**:
   A probabilidade máxima de sucesso é calculada como:
   $$
   f(k, s) = \max \left( p_k \cdot f(k+1, \text{new\_state}) \right)
   $$


In [3]:
import numpy as np

def maximize_probability(w, N, c_k, lifelines, p_k_star):
    """
    Computes the maximum probability of reaching and correctly answering a specific question (w)
    using dynamic programming.
    """
    # Initialize a 2D array to store maximum probabilities for each state and question
    probabilities = np.zeros((N, 2 ** len(lifelines)))

    # Base case: If the contestant is already at question `w`, the probability of success is 1
    for state in range(2 ** len(lifelines)):
        probabilities[w, state] = 1

    # Backward induction to compute probabilities for earlier questions
    for k in range(w - 1, 0, -1):
        for state in range(2 ** len(lifelines)):
            # Calculate probability of answering correctly without using any lifelines
            p_no_lifeline = p_k_star(k)

            # Identify the lifelines available in the current state
            available_lifelines = [lifelines[i] for i in range(len(lifelines)) if (state >> i) & 1]

            # Base probability (without lifelines)
            max_prob = p_no_lifeline * probabilities[k + 1, state]

            # Evaluate probabilities for each combination of available lifelines
            for lifeline in available_lifelines:
                new_state = state & ~(1 << lifelines.index(lifeline))  # Mark the lifeline as used
                p_with_lifeline = adjust_probability(p_no_lifeline, [lifeline], k)
                max_prob = max(max_prob, p_with_lifeline * probabilities[k + 1, new_state])

            # Update the maximum probability for the current state
            probabilities[k, state] = max_prob

    return probabilities


def print_comparison(probabilities, lifelines, w_max):
    """
    Prints a comparison of the maximum probabilities and optimal strategies
    for reaching and correctly answering each question.
    """
    print(f"{'QI':<4} {'Probability of Success':<30} {'Optimal Strategy':<30}")
    print("-" * 70)

    for w in range(1, w_max + 1):
        # Get the maximum probability for question `w`
        max_prob = max(probabilities[w, :])

        # Find the state associated with the maximum probability
        optimal_state = np.argmax(probabilities[w, :])

        # Determine which lifelines were used in the optimal state
        used_lifelines = []
        for i, lifeline in enumerate(lifelines):
            if not ((optimal_state >> i) & 1):  # If the lifeline was used, its bit is 0
                used_lifelines.append(lifeline)

        # Format the optimal strategy as a readable string
        strategy = " | ".join(used_lifelines) if used_lifelines else "no lifelines"

        # Print the results for question `w`
        print(f"{w:<4} {max_prob:<30.5f} {strategy:<30}")

In [4]:
probabilities = maximize_probability(w=5, N=16, c_k=c_k, lifelines=lifelines, p_k_star=p_k_star)
print_comparison(probabilities, lifelines, w_max=15)

QI   Probability of Success         Optimal Strategy              
----------------------------------------------------------------------
1    0.80504                        no lifelines                  
2    0.80827                        no lifelines                  
3    0.84416                        audience                      
4    0.91255                        50:50 | audience              
5    1.00000                        50:50 | phone | audience      
6    0.00000                        50:50 | phone | audience      
7    0.00000                        50:50 | phone | audience      
8    0.00000                        50:50 | phone | audience      
9    0.00000                        50:50 | phone | audience      
10   0.00000                        50:50 | phone | audience      
11   0.00000                        50:50 | phone | audience      
12   0.00000                        50:50 | phone | audience      
13   0.00000                        50:50 | phone | audien